In [6]:
import sys
import os

import sqlite3

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import hex2color
import seaborn as sns

sys.path.append(os.environ['OC_PATH'] + '01-Projets/conf')

from conf import ci_var, palette, conf_graph

In [ ]:
database = 'data/database.sqlite'
conn = sqlite3.connect(database)

tables=pd.read_sql(""" SELECT * FROM sqlite_master WHERE type='table'; """, conn)
tables

In [ ]:
df_seq = pd.read_sql(""" SELECT * FROM sqlite_sequence """, conn)
df_seq

In [ ]:
df_playattr = pd.read_sql(""" SELECT * FROM player_attributes """, conn)
df_playattr

In [ ]:
df_playattr.sort_values('date').groupby('player_fifa_api_id').tail(1)

In [ ]:
df_playattr.dtypes

In [ ]:
df_playattr.loc[df_playattr['player_fifa_api_id'] == 158023].iloc[:, 10:35]

In [ ]:
df_player = pd.read_sql(""" SELECT * FROM player """, conn)
df_player

In [ ]:
bb = pd.read_sql(""" SELECT player_fifa_api_id, player_name FROM Player """, conn)

In [ ]:
bb

In [ ]:
df_playattr.merge(bb, on='player_fifa_api_id')

In [ ]:
df_match = pd.read_sql(""" SELECT * FROM match """, conn)
df_match

In [ ]:
pd.set_option('display.max_rows', 150)
df_match.loc[1900, 'goal']

In [ ]:
df_league = pd.read_sql(""" SELECT * FROM league """, conn)
df_league

In [ ]:
df_country = pd.read_sql(""" SELECT * FROM country """, conn)
df_country

In [ ]:
df_team = pd.read_sql(""" SELECT * FROM team """, conn)
df_team

In [ ]:
df_team_attributes = pd.read_sql(""" SELECT * FROM team_attributes """, conn)
df_team_attributes

In [ ]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

# Connect to the existing database
engine = create_engine('sqlite:///data/database.sqlite')

# Reflect the database schema to generate models
metadata = MetaData()
metadata.reflect(bind=engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# Access the generated models
Player = metadata.tables['Player']

# Query for users
players = session.query(Player).all()
for player in players:
    print(player)

In [7]:
import pandas as pd
from sqlalchemy import create_engine

# Connect to the existing database
engine = create_engine('sqlite:///data/database.sqlite')

# Reflect the database schema to generate models
metadata = MetaData()
metadata.reflect(bind=engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# Access the generated models
Player = metadata.tables['Player']

# Read the query results directly into a DataFrame
df = pd.read_sql(session.query(User).statement, engine)

# Now you have your query results in a Pandas DataFrame
df

NameError: name 'MetaData' is not defined

In [8]:
def load_data():
    """Charge les données."""
    database = 'data/database.sqlite'
    conn = sqlite3.connect(database)

    df_player = pd.read_sql(""" SELECT * FROM Player_Attributes """, conn)
    df_player = df_player.sort_values('date').groupby('player_fifa_api_id').tail(1)

    df_playernames = pd.read_sql(""" SELECT player_fifa_api_id, player_name, birthday, height, weight FROM Player """, conn)
    df_player = df_player.merge(df_playernames, on='player_fifa_api_id')
    del df_playernames

    return df_player

In [9]:
df_player = load_data()

In [15]:
from datetime import datetime

df_player['birthday'] = pd.to_datetime(df_player['birthday'])

# Calculate the age
df_player['age'] = datetime.today() - df_player['birthday']

# Extract years, months, and days if needed
df_player['years'] = df_player['age'].dt.days // 365

In [16]:
df_player

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,gk_handling,gk_kicking,gk_positioning,gk_reflexes,player_name,birthday,height,weight,age,years
0,144774,174009,359194,2007-08-30 00:00:00,56.0,70.0,right,None,_0,34.0,...,23.0,40.0,23.0,23.0,Quini,1985-10-29,180.34,159,13847 days 17:13:26.206601,37
1,80571,150550,33688,2007-08-30 00:00:00,66.0,74.0,right,None,_0,25.0,...,67.0,65.0,64.0,73.0,Jeremy Gavanon,1983-09-20,182.88,163,14617 days 17:13:26.206601,40
2,115892,8038,30597,2008-08-30 00:00:00,68.0,76.0,right,None,o,69.0,...,22.0,73.0,22.0,22.0,Martijn Meerdink,1976-09-15,180.34,172,17178 days 17:13:26.206601,47
3,13938,148853,26678,2008-08-30 00:00:00,59.0,64.0,left,None,o,33.0,...,24.0,47.0,24.0,24.0,Angelo Martha,1982-04-29,185.42,181,15126 days 17:13:26.206601,41
4,140698,152782,97463,2008-08-30 00:00:00,67.0,69.0,right,None,o,72.0,...,25.0,59.0,25.0,25.0,Paulinho,1983-08-24,172.72,148,14644 days 17:13:26.206601,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11055,57230,197948,184521,2016-07-07 00:00:00,73.0,77.0,right,medium,high,51.0,...,15.0,15.0,12.0,7.0,Florian Lejeune,1991-05-20,187.96,179,11818 days 17:13:26.206601,32
11056,1330,231638,495841,2016-07-07 00:00:00,72.0,84.0,right,medium,medium,49.0,...,14.0,13.0,8.0,6.0,Adalberto Penaranda,1997-05-31,182.88,172,9615 days 17:13:26.206601,26
11057,153509,172555,45400,2016-07-07 00:00:00,68.0,70.0,right,medium,low,33.0,...,11.0,6.0,9.0,14.0,Ronnie Schwartz,1989-08-29,182.88,176,12447 days 17:13:26.206601,34
11058,19676,166074,210065,2016-07-07 00:00:00,72.0,75.0,right,medium,low,67.0,...,7.0,8.0,7.0,11.0,Bebe,1990-07-12,190.50,183,12130 days 17:13:26.206601,33


In [ ]:
df_player = df_player.dropna()

In [ ]:
df_player.columns[:9]

In [ ]:
player_info = df_player.loc[df_player['player_name'].str.contains('Lionel Messi')]

In [ ]:
player_info['reactions']

In [ ]:
import plotly.figure_factory as ff
import plotly.graph_objects as go

fig = ff.create_distplot(
    [df_player['crossing']], ['Distribution of Crossing'],
    bin_size=5,
    show_rug=False, show_hist=False
)

# Get the data from the plot
hist_data = fig['data'][0]['x']
hist_y = fig['data'][0]['y']

# Define the target y value you want to find
target_x = df_player.loc[df_player['player_name'].str.contains('Lionel Messi'), 'crossing'].values[0]

# Iterate through the histogram data to find the x coordinate for the target y value
y_coordinate = None
for i in range(len(hist_data)):
    if hist_data[i] >= target_x:
        y_coordinate = hist_y[i]
        break

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=[target_x],
        y=[y_coordinate],
        marker=dict(
            color='LightSkyBlue',
            size=20,
            line=dict(
                color='MediumPurple',
                width=2
            )
        ),
        showlegend=False
    )
)

fig.update_layout(xaxis_range=[0, 100], template="simple_white")

fig.show()

In [ ]:
hist_data

In [ ]:
df_player.loc[df_player['crossing'] == 6, 'player_name']

In [ ]:
import plotly.express as px
import pandas as pd

# Sample data
data = pd.DataFrame({
    'x': [1, 2, 3, 4, 5],
    'y': [10, 20, 15, 30, 25]
})

# Create a scatter plot using Plotly Express
fig = px.scatter(data, x='x', y='y', title='Simple Scatter Plot')

# Show the plot
fig.show()

In [ ]:
a = {
    'a': 1,
    'b': 2,
    'c': 3
}

In [ ]:
list(a.keys())[0]

In [5]:
import plotly.graph_objs as go
import numpy as np
import scipy.stats as stats

# Sample data
data = np.random.normal(size=100)

# Create a KDE curve
kde_x = np.linspace(min(data), max(data), 1000)
kde_y = stats.gaussian_kde(data)(kde_x)

# Create a scatter plot for the KDE curve
kde_trace = go.Scatter(x=kde_x, y=kde_y, mode='lines', name="KDE")

# Create a figure
fig = go.Figure(data=kde_trace)

# Customize hovertemplate
hovertemplate = "Value: %{x:.2f}<br>Density: %{y:.2f}"
kde_trace.update(hoverinfo="x+y", hovertemplate=hovertemplate)

# Layout customization
fig.update_layout(
    title="KDE Distribution Plot",
    xaxis_title="Value",
    yaxis_title="Density",
    template='simple_white'
)

# Show the plot
fig.show()

In [24]:
df_player = df_player.dropna()

In [38]:
df_player.iloc[:, 9:-6]

,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
479,62.0,41.0,49.0,57.0,57.0,49.0,39.0,46.0,49.0,63.0,...,62.0,48.0,54.0,62.0,58.0,15.0,12.0,6.0,11.0,5.0
480,52.0,64.0,79.0,68.0,35.0,39.0,49.0,44.0,66.0,63.0,...,71.0,54.0,73.0,74.0,72.0,10.0,8.0,11.0,11.0,6.0
481,52.0,46.0,53.0,66.0,39.0,61.0,53.0,42.0,61.0,67.0,...,68.0,46.0,64.0,65.0,61.0,5.0,15.0,5.0,15.0,7.0
482,50.0,45.0,51.0,64.0,50.0,52.0,49.0,61.0,65.0,63.0,...,69.0,54.0,59.0,63.0,58.0,14.0,12.0,13.0,10.0,10.0
483,58.0,48.0,54.0,61.0,56.0,61.0,41.0,47.0,64.0,53.0,...,43.0,44.0,51.0,49.0,36.0,15.0,10.0,15.0,6.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11055,51.0,42.0,75.0,68.0,42.0,47.0,47.0,41.0,67.0,62.0,...,32.0,43.0,74.0,75.0,69.0,11.0,15.0,15.0,12.0,7.0
11056,49.0,78.0,71.0,63.0,56.0,74.0,45.0,43.0,49.0,73.0,...,59.0,77.0,20.0,20.0,21.0,6.0,14.0,13.0,8.0,6.0
11057,33.0,70.0,56.0,63.0,60.0,63.0,55.0,27.0,56.0,66.0,...,45.0,66.0,23.0,17.0,23.0,13.0,11.0,6.0,9.0,14.0
11058,67.0,67.0,57.0,66.0,69.0,80.0,67.0,73.0,58.0,77.0,...,61.0,66.0,19.0,24.0,21.0,15.0,7.0,8.0,7.0,11.0


In [25]:
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=2, radius=0.4)
neigh.fit(df_player.iloc[:, 9:-6])

NearestNeighbors(n_neighbors=2, radius=0.4)

In [37]:
df_player.loc[5047][9:-6]

crossing              72.0
finishing             66.0
heading_accuracy      58.0
short_passing         75.0
volleys               74.0
dribbling             73.0
curve                 64.0
free_kick_accuracy    67.0
long_passing          73.0
ball_control          74.0
acceleration          62.0
sprint_speed          54.0
agility               75.0
reactions             72.0
balance               74.0
shot_power            70.0
jumping               53.0
stamina               55.0
strength              66.0
long_shots            74.0
aggression            64.0
interceptions         60.0
positioning           71.0
vision                72.0
penalties             69.0
marking               43.0
standing_tackle       58.0
sliding_tackle        55.0
gk_diving              8.0
gk_handling           13.0
gk_kicking             6.0
gk_positioning        12.0
gk_reflexes            8.0
Name: 5047, dtype: object

In [34]:
df_player.loc[5699][9:-6]

crossing              60.0
finishing             25.0
heading_accuracy      77.0
short_passing         69.0
volleys               55.0
dribbling             52.0
curve                 41.0
free_kick_accuracy    42.0
long_passing          68.0
ball_control          70.0
acceleration          65.0
sprint_speed          74.0
agility               58.0
reactions             60.0
balance               60.0
shot_power            62.0
jumping               66.0
stamina               51.0
strength              80.0
long_shots            36.0
aggression            84.0
interceptions         66.0
positioning           54.0
vision                55.0
penalties             45.0
marking               75.0
standing_tackle       79.0
sliding_tackle        78.0
gk_diving             13.0
gk_handling           13.0
gk_kicking             7.0
gk_positioning         7.0
gk_reflexes            8.0
Name: 5699, dtype: object

In [51]:
neigh.kneighbors(df_player.loc[df_player['player_name'].str.contains('Lionel Messi')].iloc[:, 9:-6].values, 2, return_distance=False)

/home/tristan/Programs/anaconda3/envs/default/lib/python3.9/site-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but NearestNeighbors was fitted with feature names



array([[4745, 5231]])

In [48]:
df_player.loc[df_player['player_name'].str.contains('Lionel Messi')].iloc[:, 9:-6].values

array([[80., 93., 71., 88., 85., 96., 89., 90., 79., 96., 95., 90., 92.,
        92., 95., 80., 68., 75., 59., 88., 48., 22., 90., 90., 74., 13.,
        23., 21.,  6., 11., 15., 14.,  8.]])

In [61]:
pd.DataFrame(df_player.iloc[4745]).T

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,gk_handling,gk_kicking,gk_positioning,gk_reflexes,player_name,birthday,height,weight,age,years
5397,102483,158023,30981,2015-12-17 00:00:00,94.0,94.0,left,medium,low,80.0,...,11.0,15.0,14.0,8.0,Lionel Messi,1987-06-24,170.18,159,13244 days 17:13:26.206601,36


In [47]:
df_player.loc[5699][9:-6]

crossing              60.0
finishing             25.0
heading_accuracy      77.0
short_passing         69.0
volleys               55.0
dribbling             52.0
curve                 41.0
free_kick_accuracy    42.0
long_passing          68.0
ball_control          70.0
acceleration          65.0
sprint_speed          74.0
agility               58.0
reactions             60.0
balance               60.0
shot_power            62.0
jumping               66.0
stamina               51.0
strength              80.0
long_shots            36.0
aggression            84.0
interceptions         66.0
positioning           54.0
vision                55.0
penalties             45.0
marking               75.0
standing_tackle       79.0
sliding_tackle        78.0
gk_diving             13.0
gk_handling           13.0
gk_kicking             7.0
gk_positioning         7.0
gk_reflexes            8.0
Name: 5699, dtype: object

In [62]:
df_player.iloc[:, 9:-6]

,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
479,62.0,41.0,49.0,57.0,57.0,49.0,39.0,46.0,49.0,63.0,...,62.0,48.0,54.0,62.0,58.0,15.0,12.0,6.0,11.0,5.0
480,52.0,64.0,79.0,68.0,35.0,39.0,49.0,44.0,66.0,63.0,...,71.0,54.0,73.0,74.0,72.0,10.0,8.0,11.0,11.0,6.0
481,52.0,46.0,53.0,66.0,39.0,61.0,53.0,42.0,61.0,67.0,...,68.0,46.0,64.0,65.0,61.0,5.0,15.0,5.0,15.0,7.0
482,50.0,45.0,51.0,64.0,50.0,52.0,49.0,61.0,65.0,63.0,...,69.0,54.0,59.0,63.0,58.0,14.0,12.0,13.0,10.0,10.0
483,58.0,48.0,54.0,61.0,56.0,61.0,41.0,47.0,64.0,53.0,...,43.0,44.0,51.0,49.0,36.0,15.0,10.0,15.0,6.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11055,51.0,42.0,75.0,68.0,42.0,47.0,47.0,41.0,67.0,62.0,...,32.0,43.0,74.0,75.0,69.0,11.0,15.0,15.0,12.0,7.0
11056,49.0,78.0,71.0,63.0,56.0,74.0,45.0,43.0,49.0,73.0,...,59.0,77.0,20.0,20.0,21.0,6.0,14.0,13.0,8.0,6.0
11057,33.0,70.0,56.0,63.0,60.0,63.0,55.0,27.0,56.0,66.0,...,45.0,66.0,23.0,17.0,23.0,13.0,11.0,6.0,9.0,14.0
11058,67.0,67.0,57.0,66.0,69.0,80.0,67.0,73.0,58.0,77.0,...,61.0,66.0,19.0,24.0,21.0,15.0,7.0,8.0,7.0,11.0


In [64]:
df = pd.DataFrame(
    [
        [30, 40, 35],
        [65, 67, 78],
        [43, 60, 52]
    ],
    columns=['skill1', 'skill2', 'skill3']
)
df

,skill1,skill2,skill3
0,30,40,35
1,65,67,78
2,43,60,52


In [68]:
df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda row: row / max(row), axis=1)

In [69]:
df

,skill1,skill2,skill3
0,30,1.000000,0.875000
1,65,0.858974,1.000000
2,43,1.000000,0.866667
